# Chapter 2 - Working with text data

# Downloading data

```python
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "../datasets/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)
```

In [1]:
import re
import tiktoken

In [2]:
with open(file="../datasets/the-verdict.txt", mode="r", encoding="utf-8") as f:
    raw_text = f.read()

In [3]:
print(f"Total number of character: {len(raw_text)}")
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
text = "Hello, world. This, is a test."
result = re.split(r"(\s)", text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result = re.split(r"([,.]|\s)", text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [7]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [11]:
vocab = {token: idx for idx, token in enumerate(all_words)}

In [ ]:
from __future__ import annotations

from typing import List


class SimpleTokenizerV1:

    def __init__(self, vocab: dict) -> SimpleTokenizerV1:

        self.vocab = vocab
        self.idx_to_token = {idx: token for token, idx in vocab.items()}

    def encode(self, text: List[str]) -> List[int]:

        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.vocab[s] for s in preprocessed]

        return ids
    
    def decode(self, ids: List[int]) -> str:

        text = " ".join([self.idx_to_token[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab=vocab)

In [14]:
text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""

In [15]:
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [17]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: idx for idx, token in enumerate(all_tokens)}

In [ ]:
from __future__ import annotations

import re

from typing import List


class SimpleTokenizerV2:

    def __init__(self, vocab: dict) -> SimpleTokenizerV1:

        self.vocab = vocab
        self.idx_to_token = {idx: token for token, idx in vocab.items()}

    def encode(self, text: List[str]) -> List[int]:

        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.vocab 
								        else "<|unk|>" for item in preprocessed]
        ids = [self.vocab[s] for s in preprocessed]

        return ids
    
    def decode(self, ids: List[int]) -> str:

        text = " ".join([self.idx_to_token[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [20]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [21]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [22]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [23]:
tokenizer = tiktoken.get_encoding("gpt2")

In [24]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text=text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [25]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [26]:
text = "Akwirw ier"
integers = tokenizer.encode(text=text)

In [27]:
strings = tokenizer.decode(integers)
print(strings)

Akwirw ier


In [28]:
enc_text = tokenizer.encode(raw_text)

In [29]:
from __future__ import annotations


import torch

from torch.utils.data import Dataset, DataLoader


class GPTDataset(Dataset):

    def __init__(
            self, 
            tokenizer, 
            txt: str, 
            max_legnth: int, 
            stride: int
    ) -> GPTDataset:
        
        super().__init__()
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_legnth, stride):
            input_chunk = token_ids[i:i+max_legnth]
            target_chunk = token_ids[i+1:i+max_legnth+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [30]:
def create_dataloader(
        txt: str, 
        batch_size: int=4, 
        max_length: int=256, 
        stride: int=128, 
        shuffle: bool=True, 
        drop_last: bool=True, 
        num_workers: int=0
    ) -> DataLoader:

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(
        tokenizer=tokenizer, 
        txt=txt, 
        max_legnth=max_length, 
        stride=stride
    )
    dataloader = DataLoader(
        dataset=dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last, 
        num_workers=num_workers
    )

    return dataloader

In [31]:
dataloader = create_dataloader(
    txt=raw_text, 
    batch_size=1, 
    max_length=4, 
    stride=1, 
    shuffle=False
)
data_iter = iter(dataloader)
first_batch = next(data_iter)

In [32]:
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [33]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

In [34]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [35]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [36]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader(
	raw_text, 
	batch_size=8,
	max_length=max_length,
	stride=max_length,
	shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
token_embeddings = token_embedding_layer(inputs)

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))

input_embeddings = token_embeddings + pos_embeddings

In [38]:
pos_embeddings.shape

torch.Size([4, 256])

In [37]:
input_embeddings.shape

torch.Size([8, 4, 256])